In [67]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [123]:
books = pd.read_csv('datasets/Books.csv')
ratings = pd.read_csv('datasets/Ratings.csv')
users = pd.read_csv('datasets/Users.csv')

/var/folders/rb/2synj62156l50xph3x3mq46m0000gn/T/ipykernel_41265/2326110046.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('datasets/Books.csv')


In [108]:
books.dropna(inplace=True)
books['Year-Of-Publication'] = books['Year-Of-Publication'].astype('int64')
dataset = pd.merge(ratings, books, on="ISBN")
dataset.drop(columns=['Image-URL-S','Image-URL-M','Image-URL-L'],inplace=True)

In [109]:
dataset['Book-Rating'] /= 2
dataset = dataset[dataset['Book-Rating'] != 0]
dataset['Book-Rating'] -= 3

In [110]:
popularity_threshold = 10
data = (dataset.groupby(by = ['Book-Title'])['Book-Rating'].count().reset_index().
        rename(columns = {'Book-Rating': 'Total-Rating'})[['Book-Title', 'Total-Rating']])

result = pd.merge(data, dataset, on='Book-Title')
result = result[result['Total-Rating'] >= popularity_threshold]
result = result.reset_index(drop = True)

matrix = result.pivot_table(index = 'Book-Title', columns = 'User-ID', values = 'Book-Rating').fillna(0)
up_matrix = csr_matrix(matrix)

In [130]:
model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model.fit(up_matrix)
def recommend_books(user_ratings: dict, k: int) -> list:
    avg_embedding = np.zeros((up_matrix.shape[1],))
    for book, rating in user_ratings.items():
        avg_embedding += matrix.loc[book].values * (rating-3)
    avg_embedding /= len(user_ratings)
    distances, indices = model.kneighbors(avg_embedding.reshape(1, -1), n_neighbors = k+1)
    result = []
    for i in range(0, len(distances.flatten())):
        if i > 0:
            name = matrix.index[indices.flatten()[i]]
            if name not in user_ratings:
                result.append(name)
    return result

In [132]:
user_ratings = {'The Hobbit: or There and Back Again': 4,
                 "Harry Potter and the Sorcerer's Stone (Book 1)": 5}
recommended_number = 10

print(*recommend_books(user_ratings, recommended_number), sep='\n')


Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Order of the Phoenix (Book 5)
The Return of the King (The Lord of The Rings, Part 3)
The Two Towers (The Lord of the Rings, Part 2)
The Fellowship of the Ring (The Lord of the Rings, Part 1)
The Black Cauldron (Chronicles of Prydain (Paperback))
Dune Messiah (Dune Chronicles, Book 2)
